In [4]:
import pandas as pd
import numpy as np
import plotly.express as px
import numpy as np
from sklearn.preprocessing import StandardScaler
from typing import Tuple, List, Callable

In [3]:
train = pd.read_csv('train.csv')
train

,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,Panel_Temperature_dwd_std,Std_Temperature_dwd,Mean_Temperature_dwd,cos_hour,cos_day,solar_mw_lag_24h,capacity_mwp,Solar_MWh_credit
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189078,0.000501,12.391174,0.695381,0.734405,12.346043,1.000000,-0.150055,0.0,2130.749513,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189162,0.000578,12.274750,0.802515,0.892851,12.229531,1.000000,-0.150055,0.0,2130.749513,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189246,0.000665,12.158325,0.923509,1.057795,12.113020,0.965926,-0.150055,0.0,2130.749513,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189333,0.000723,12.038016,1.004165,1.175842,11.986561,0.965926,-0.150055,0.0,2130.749513,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.189419,0.000789,11.917706,1.096040,1.295864,11.860103,0.866025,-0.150055,0.0,2130.749513,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50882,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,0.581458,0.717884,13.779720,0.965926,-0.591261,0.0,2454.550187,0.0
50883,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,0.552118,0.658103,13.422141,1.000000,-0.577292,0.0,2454.550187,0.0
50884,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,0.536205,0.593122,13.101896,1.000000,-0.577292,0.0,2454.550187,0.0
50885,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,0.529482,0.534463,12.781651,0.965926,-0.577292,0.0,2454.550187,0.0


In [7]:
X = train.drop(columns=['Solar_MWh_credit'])
y = train['Solar_MWh_credit']

In [10]:
from sklearn.preprocessing import PolynomialFeatures

# Preprocess X to include squared terms (and possibly higher-order terms)
poly = PolynomialFeatures(degree=2, include_bias=False)
X_augmented = poly.fit_transform(X)

In [12]:
class GeneticAlgorithm:
    def __init__(
        self,
        pop_size: int = 100,
        n_generations: int = 1000,
        mutation_rate: float = 0.01,
        crossover_rate: float = 0.8
    ):
        self.pop_size = pop_size
        self.n_generations = n_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.best_solution = None
        self.best_fitness = float('-inf')
    
    def initialize_population(self, n_features: int) -> np.ndarray:
        return np.random.randn(self.pop_size, n_features)
    
    def fitness_function(self, individual: np.ndarray, X: np.ndarray, y: np.ndarray) -> float:
        predictions = np.sum(X * individual, axis=1)
        mse = np.mean((predictions - y) ** 2)
        return -mse  # Negative because we want to maximize fitness
    
    def select_parents(self, population: np.ndarray, fitnesses: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        # Tournament selection
        idx1, idx2 = np.random.randint(0, self.pop_size, size=(2, 2))
        parent1 = population[idx1[0]] if fitnesses[idx1[0]] > fitnesses[idx1[1]] else population[idx1[1]]
        parent2 = population[idx2[0]] if fitnesses[idx2[0]] > fitnesses[idx2[1]] else population[idx2[1]]
        return parent1, parent2
    
    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        if np.random.random() < self.crossover_rate:
            crossover_point = np.random.randint(1, len(parent1))
            child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            return child1, child2
        return parent1.copy(), parent2.copy()
    
    def mutate(self, individual: np.ndarray) -> np.ndarray:
        mutation_mask = np.random.random(individual.shape) < self.mutation_rate
        individual[mutation_mask] += np.random.normal(0, 0.1, size=np.sum(mutation_mask))
        return individual
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        n_features = X.shape[1]
        population = self.initialize_population(n_features)
        
        for generation in range(self.n_generations):
            new_population = []
            fitnesses = np.array([self.fitness_function(ind, X, y) for ind in population])
            
            # Keep track of best solution
            best_idx = np.argmax(fitnesses)
            if fitnesses[best_idx] > self.best_fitness:
                self.best_fitness = fitnesses[best_idx]
                self.best_solution = population[best_idx].copy()
            
            while len(new_population) < self.pop_size:
                parent1, parent2 = self.select_parents(population, fitnesses)
                child1, child2 = self.crossover(parent1, parent2)
                child1, child2 = self.mutate(child1), self.mutate(child2)
                new_population.extend([child1, child2])
            
            population = np.array(new_population[:self.pop_size])
            
            if (generation + 1) % 10 == 0:
                print(f"Generation {generation + 1}, Best Fitness: {self.best_fitness}")
        
        return self.best_solution


In [15]:
# Train genetic algorithm
ga = GeneticAlgorithm(pop_size=100, n_generations=1000)
best_solution = ga.fit(X_augmented, y)
print(best_solution)
#print error
predictions = np.sum(X_augmented * best_solution, axis=1)
mse = np.mean((predictions - y) ** 2)
print(f'MSE: {mse}')

Generation 10, Best Fitness: -80199424645.66724
Generation 20, Best Fitness: -28650217071.01883
Generation 30, Best Fitness: -28650217071.01883
Generation 40, Best Fitness: -28650217071.01883
Generation 50, Best Fitness: -28650217071.01883
Generation 60, Best Fitness: -12986458607.263786
Generation 70, Best Fitness: -8877287765.536222
Generation 80, Best Fitness: -7775943795.654055
Generation 90, Best Fitness: -5881237262.858472
Generation 100, Best Fitness: -3538598773.14335
Generation 110, Best Fitness: -3325448987.6089115
Generation 120, Best Fitness: -3221214940.827772
Generation 130, Best Fitness: -3131798313.9973598
Generation 140, Best Fitness: -2964362252.7155395
Generation 150, Best Fitness: -2964362252.7155395
Generation 160, Best Fitness: -2964362252.7155395
Generation 170, Best Fitness: -2948204167.115789
Generation 180, Best Fitness: -2885364808.967398
Generation 190, Best Fitness: -2680345115.129241
Generation 200, Best Fitness: -2602994709.765782
Generation 210, Best Fit

In [ ]:


class GeneticAlgorithm:
    def __init__(
        self,
        pop_size: int = 100,
        n_generations: int = 100,
        mutation_rate: float = 0.01,
        crossover_rate: float = 0.8
    ):
        self.pop_size = pop_size
        self.n_generations = n_generations
        self.mutation_rate = mutation_rate
        self.crossover_rate = crossover_rate
        self.best_solution = None
        self.best_fitness = float('-inf')
    
    def initialize_population(self, n_features: int) -> np.ndarray:
        return np.random.randn(self.pop_size, n_features)
    
    def fitness_function(self, individual: np.ndarray, X: np.ndarray, y: np.ndarray) -> float:
        predictions = np.sum(X * individual, axis=1)
        mse = np.mean((predictions - y) ** 2)
        return -mse  # Negative because we want to maximize fitness
    
    def select_parents(self, population: np.ndarray, fitnesses: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        # Tournament selection
        idx1, idx2 = np.random.randint(0, self.pop_size, size=(2, 2))
        parent1 = population[idx1[0]] if fitnesses[idx1[0]] > fitnesses[idx1[1]] else population[idx1[1]]
        parent2 = population[idx2[0]] if fitnesses[idx2[0]] > fitnesses[idx2[1]] else population[idx2[1]]
        return parent1, parent2
    
    def crossover(self, parent1: np.ndarray, parent2: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        if np.random.random() < self.crossover_rate:
            crossover_point = np.random.randint(1, len(parent1))
            child1 = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child2 = np.concatenate([parent2[:crossover_point], parent1[crossover_point:]])
            return child1, child2
        return parent1.copy(), parent2.copy()
    
    def mutate(self, individual: np.ndarray) -> np.ndarray:
        mutation_mask = np.random.random(individual.shape) < self.mutation_rate
        individual[mutation_mask] += np.random.normal(0, 0.1, size=np.sum(mutation_mask))
        return individual
    
    def fit(self, X: np.ndarray, y: np.ndarray) -> np.ndarray:
        n_features = X.shape[1]
        population = self.initialize_population(n_features)
        
        for generation in range(self.n_generations):
            new_population = []
            fitnesses = np.array([self.fitness_function(ind, X, y) for ind in population])
            
            # Keep track of best solution
            best_idx = np.argmax(fitnesses)
            if fitnesses[best_idx] > self.best_fitness:
                self.best_fitness = fitnesses[best_idx]
                self.best_solution = population[best_idx].copy()
            
            while len(new_population) < self.pop_size:
                parent1, parent2 = self.select_parents(population, fitnesses)
                child1, child2 = self.crossover(parent1, parent2)
                child1, child2 = self.mutate(child1), self.mutate(child2)
                new_population.extend([child1, child2])
            
            population = np.array(new_population[:self.pop_size])
            
            if (generation + 1) % 10 == 0:
                print(f"Generation {generation + 1}, Best Fitness: {self.best_fitness}")
        
        return self.best_solution

# Example usage
def prepare_data(X: np.ndarray, y: np.ndarray) -> Tuple[np.ndarray, np.ndarray, StandardScaler]:
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    return X_scaled, y, scaler

def main():
    # Generate sample data
    np.random.seed(42)
    X = np.random.randn(1000, 5)  # 1000 samples, 5 features
    true_coefficients = np.array([1, -0.5, 2, 0.8, -1.2])
    y = np.dot(X, true_coefficients) + np.random.normal(0, 0.1, 1000)
    
    # Prepare data
    X_scaled, y, scaler = prepare_data(X, y)
    
    # Train genetic algorithm
    ga = GeneticAlgorithm(pop_size=100, n_generations=100)
    best_solution = ga.fit(X_scaled, y)
    
    print("True coefficients:", true_coefficients)
    print("Estimated coefficients:", best_solution)

if __name__ == "__main__":
    main()